In [1]:
%load_ext autoreload

In [3]:
import numpy as np
import os
import mcubes
import meshcat
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint
from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform, RotationMatrix
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions, Rgba
import time
from meshcat import Visualizer
from functools import partial

import ipywidgets as widgets
from IPython.display import display

#from meshcat_cpp_utils import StartMeshcat, MeshcatJointSliders
from pydrake.all import MeshcatVisualizerCpp, MeshcatVisualizerParams, Role
import rrt, utils# spp

In [4]:
import spp


ImportError: cannot import name 'BsplineTrajectory_' from 'pydrake.trajectories' (/home/peter/.cache/bazel/_bazel_peter/76bc0000d64a792547034bb2ef203b82/execroot/drake/bazel-out/k8-opt/bin/sandbox/iiwa_example.runfiles/drake/bindings/pydrake/trajectories.cpython-38-x86_64-linux-gnu.so)

In [6]:
from pydr

ImportError: cannot import name 'BsplineTrajectory_' from 'pydrake.trajectories' (/home/peter/.cache/bazel/_bazel_peter/76bc0000d64a792547034bb2ef203b82/execroot/drake/bazel-out/k8-opt/bin/sandbox/iiwa_example.runfiles/drake/bindings/pydrake/trajectories.cpython-38-x86_64-linux-gnu.so)

In [4]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

In [24]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()

#model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/urdf/iiwa14_polytope_collision.urdf")
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
#model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
#model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf")

#box_file = FindResourceOrThrow("drake/sandbox/assets/box.urdf")
box_file_1 = FindResourceOrThrow("drake/sandbox/shelves.sdf")
box_file_2 = FindResourceOrThrow("drake/sandbox/shelves2.sdf")
box_file_3 = FindResourceOrThrow("drake/sandbox/shelves3.sdf")
box_file_4 = FindResourceOrThrow("drake/sandbox/shelves4.sdf")

models =[]

builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))

models.append(Parser(plant, scene_graph).AddModelFromFile(box_file_1))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file_2))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file_3))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file_4))

sp = 0.4
locs = [ [0,0,0], [sp, 1.4*sp,0.4], [sp,-1.4*sp,0.4], [-sp,-1.4*sp,0.4], [-sp,1.4*sp,0.4]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(), RigidTransform(locs[idx]))
    idx +=1
# plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform(locs[idx]))

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False, )

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)



You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6005...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/
Connected to meshcat-server.


In [6]:
sliders = []
for joint_idx in range(1,8):
    sliders.append(
        widgets.FloatSlider(
        min=plant.GetJointByName(f"iiwa_joint_{joint_idx}").position_lower_limit(), 
        max=plant.GetJointByName(f"iiwa_joint_{joint_idx}").position_upper_limit(), 
        value=0, 
        description=f'iiwa_joint_{joint_idx}'
        )
    )

ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.001)

def eval_cons(q, c, tol):
    return 1-1*float(c.evaluator().CheckSatisfied(q, tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)
    
def showres(q):
    for joint_idx in range(1,8):
        plant.GetJointByName(f"iiwa_joint_{joint_idx}").set_angle(plant_context, q[joint_idx-1])
    col = col_func_handle(q)
    if col:
        vis["collision"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFF0000))
        vis["collision"].set_transform(
                meshcat.transformations.translation_matrix([0, 1.0, 1.0]))
    else:
        vis["collision"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x00FF00))
        vis["collision"].set_transform(
                meshcat.transformations.translation_matrix([0, 1.0, 1.0]))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col , end = "\r")

    
def set_joint_angles(vals):
    for idx in range(1,8):
        plant.GetJointByName(f"iiwa_joint_{joint_idx}").set_angle(plant_context, vals[joint_idx-1])
    
    
    
q_init = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# q_start = [-1.37, -1.39, -1.57, -1.19, -1.57, 0.00, 0.00]
# q_end = [1.53, -1.19, 1.83, 0.91, -1.27, -0.59, 0.00]
q = q_init.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1


In [7]:
for slider in sliders:
    display(slider)
display(vis.jupyter_cell())

FloatSlider(value=0.0, description='iiwa_joint_1', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='iiwa_joint_2', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='iiwa_joint_3', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='iiwa_joint_4', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='iiwa_joint_5', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='iiwa_joint_6', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='iiwa_joint_7', max=3.05433, min=-3.05433)

In [25]:
#problem config
start = np.array([1.43, -0.69, 0.83, 1.01, -1.97, 1.11, 0.00])
target = np.array([1.73, 1.41, 0.43, -1.19, -1.27, -1.19, 0.00])
showres(target)
limits =[plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]

In [27]:

RRT = rrt.RRT(start = target,
              goal = start,
              limits = limits,
              col_func_handle=col_func_handle,
              max_extend_length=0.1,
              extend_steps=0.005,
              init_goal_sample_rate=0.03,
              goal_sample_rate_scaler=0.6,
              verbose = True, 
              plotcallback = None)

success, path = RRT.run(30000)

it: 0 distance to target:  3.868 goalsample prob:  0.030
it: 2 distance to target:  3.863 goalsample prob:  0.030
it: 3 distance to target:  3.831 goalsample prob:  0.030
it: 11 distance to target:  3.824 goalsample prob:  0.030
it: 12 distance to target:  3.821 goalsample prob:  0.030
it: 13 distance to target:  3.767 goalsample prob:  0.030
it: 16 distance to target:  3.716 goalsample prob:  0.030
it: 17 distance to target:  3.671 goalsample prob:  0.030
it: 18 distance to target:  3.609 goalsample prob:  0.030
it: 19 distance to target:  3.566 goalsample prob:  0.030
it: 22 distance to target:  3.536 goalsample prob:  0.030
it: 23 distance to target:  3.508 goalsample prob:  0.030
it: 24 distance to target:  3.465 goalsample prob:  0.030
it: 25 distance to target:  3.456 goalsample prob:  0.030
it: 26 distance to target:  3.425 goalsample prob:  0.030
it: 27 distance to target:  3.378 goalsample prob:  0.030
it: 31 distance to target:  3.375 goalsample prob:  0.030
it: 36 distance t

In [30]:
def do_iris(q_seed, iris_options):
    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    return ellipse, hpoly

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 50
iris_options.enable_ibex = False

regions = []
ellipses = []
its = 20
seed_points = [start, target]

for point in seed_points:
    ell, reg = do_iris(point, iris_options)
    regions.append(reg)
    ellipses.append(ell)
    
for it in range(its):
    #rejection sampling to get initial feasible point 
    found = False
    while not found:
        t = np.random.rand(len(limits[0]))
        q_samp = (1-t)*limits[0] + t*limits[1]
        found = (col_func_handle(q_samp)==0.0)
    print("point found: ", q_samp)
    ell, reg = do_iris(q_samp, iris_options)
    if ell.Volume() < 1000.0:
        regions.append(reg)
        ellipses.append(ell)
        
        set_joint_angles(ell.center())
        tf = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(6)))
        mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
        mat.reflectivity = 1.0
        vis['centers']['points' + str(it)].set_object(
                    meshcat.geometry.Sphere(0.02), mat)
        vis['centers']['points' + str(it)].set_transform(tf.GetAsMatrix4())

Time:   0.66 	Volume:   0.01 	Center: [ 1.43184092 -0.50808767  0.42293069  1.01137272 -2.36231947  0.63611496
  0.30776114]
Time:   1.29 	Volume:   0.05 	Center: [ 1.47900251  1.54984112  0.32712214 -1.13712335 -0.74533392  0.3218557
  0.40648641]
point found:  [-0.60908115  0.34745994 -1.44033049  0.07438593  1.72353013  1.34432023
 -0.95040098]
Time:   3.96 	Volume: 391.20 	Center: [-5.28126876e-10  3.88104489e-02  1.79341096e-01  8.26645097e-02
  2.07433626e-10 -1.71721994e-01  2.03185335e-09]
point found:  [-0.02358567  1.42342418 -0.01975274  1.93712919  2.19772281  0.12560346
  2.08234149]
Time:   6.74 	Volume:  47.39 	Center: [ 9.30555725e-03 -1.19939584e-01 -3.07434233e-02 -2.59155428e-01
 -1.55916702e-09  1.70779645e-01  3.53397267e-09]
point found:  [-1.20828391  0.05944922 -1.98940093  1.39826645  1.7921947   1.40621706
  2.88911084]
Time:   0.68 	Volume:   0.01 	Center: [-1.38087987  0.07198577 -1.65770839  1.52870304  1.91479806  1.26236543
  2.53507717]
point found:  [ 0

In [ ]:
start_time = time.time()
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start, target, regions)
spp.set_max_velocity([.8, .8, .8,.8, .8, .8,.8])
spp.set_extra_control_points_per_region(2)

# print(spp.num_regions())
traj = spp.Solve()
print(time.time() - start_time)
print(traj.start_time())
print(traj.end_time())
#visualize
maxit = 60
pts = []
for it in range(maxit):
    pts.append(traj.value(it*traj.end_time()/maxit))
    mat = meshcat.geometry.MeshLambertMaterial(color=0xFFF812)
    mat.reflectivity = 1.0
    set_joint_angles(pts[-1].reshape(-1,))
    tf= plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(3)))
   
     
    mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
    mat.reflectivity = 1.0
    vis['traj']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.02), mat)
    vis['traj']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(tf.tranlation()))
    

In [29]:
traj= utils.PWLinTraj(path, 5.0)
substeps = 1000
utils.animate(traj, showres, substeps, 5*substeps)

In [34]:
import spp


ImportError: cannot import name 'GraphOfConvexSets' from 'pydrake.geometry.optimization' (unknown location)

In [37]:
from pydrake.geometry.optimization import 
